In [1]:
# Get pandas for dealing with xlsx
import pandas as pd
# Old numpy
import numpy as np
# Stats
import statistics as st
import statsmodels.api as sm
# Plotting
import matplotlib.pyplot as plt
import math

In [2]:
# set path
path = "/Users/chaseabram/UChiGit/Skwad/Empirical Analysis III/Problem Sets/Heckman Pset 4/PS4_Data.xlsx"
# read file
xls = pd.ExcelFile(path)

In [68]:
# Get number of sheets (4)
num_sheets = len(xls.sheet_names)
# initialize data frame
dfs = [0]*num_sheets

# results data frame
# column_names = ['sample', 
#                'betax0', 'betax1', 'betax2','betaz0', 'betaz1', 'betaz1', 'Dhat']
# results = pd.DataFrame(columns = column_names)

# read each sheet into data frame and perform analysis
for i in range(0,num_sheets):
    # Add to data frame
    sheet_name = 'DataSet' + str(i)
    dfs[i] = pd.read_excel(xls, sheet_name)
    
    # read in data
    Y = dfs[i]['Y']
    
    D = dfs[i]['D']
    
    X = np.zeros((3, len(dfs[i])))
    X[0] = [1]*len(dfs[i])
    X[1] = dfs[i]['X1']
    X[2] = dfs[i]['X2']
    print(np.shape(X[1:4]))
    
    Z = np.zeros((3, len(dfs[i])))
    Z[0] = [1]*len(dfs[i])
    Z[1] = dfs[i]['Z1']
    Z[2] = dfs[i]['Z2']
    
    # A
#     print(np.shape(np.vstack((X[1:2],Z[1:2]))))
    X_A = np.vstack((X,Z[1:3])).T
    model_A = sm.OLS(D,X_A)
    result_A = model_A.fit()
    betas = result_A.params
    print(betas)
    Dhat_A = result_A.predict(X_A)
    print(max(Dhat_A)
    
    # B
    
    
    
    
    
    
    # means
#     mu_1 = np.mean(X[1])
#     mu_2 = np.mean(X[2])
    
#     # var-cov
#     Sigma = np.cov(X)
#     sigma_1 = math.sqrt(Sigma[1,1])
#     sigma_2 = math.sqrt(Sigma[2,2])
                       
#     # rho
#     rho = Sigma[2,1]/(Sigma[1,1]*Sigma[2,2])
    
#     # run ols with X1 and X2
#     model = sm.OLS(Y,X.T)
#     result = model.fit()
#     betas = result.params
    
#     # est var of residuals
#     sigma_e = np.cov(result.resid)
    
#     # t stat
#     tstat = result.tvalues[2]
    
#     # t default is reject beta_2 = 0
#     t_test = 1
    
#     # t test for beta 2
#     if abs(tstat) < 1.964:
#         # t test cannot reject null of beta_2 = 0
#         t_test = 0
        
#         # OVB regression replaces beta_0 and beta_1
#         model = sm.OLS(Y,X[0:2].T)
#         result = model.fit()
#         betas[0:2] = result.params 
    
#     # create new data frame entry with variables
#     new_df = pd.DataFrame(np.array([i, mu_1, mu_2, sigma_1, sigma_2, sigma_e,
#                           rho, betas[0], betas[1], betas[2], tstat, t_test])).T
#     new_df.columns = column_names
    
#     # add to results data frame
#     results = pd.concat([results, new_df])

# # save estimates
# results.to_csv ("/Users/chaseabram/UChiGit/Skwad/Empirical Analysis III/Problem Sets/Heckman Pset 4/Q3_output.csv", index = False, header=True)
# results

(2, 10000)
const    0.564649
x1       0.093402
x2       0.093139
x3      -0.086389
x4       0.004562
dtype: float64
[0.5673583  0.52976565 0.72068223 ... 0.69741828 0.53866268 0.4976591 ]
(2, 1000)
const   -13.571277
x1        0.104304
x2        0.039508
x3        6.710722
x4       -0.222615
dtype: float64
[ 1.74425395e-01  3.05611430e-01  2.71263147e-01  4.40165951e-01
  6.43810363e-01  5.39885272e-01  5.70914582e-01  4.96365312e-01
  2.12829120e-01  6.59096006e-01 -9.53753232e-02  5.21478384e-01
  2.79641717e-01  5.46448815e-01  3.58252290e-01  3.10502979e-01
  3.24711062e-01  7.39984318e-02  1.37046950e-01  7.50749223e-01
  4.55837880e-01  4.35845017e-01  4.26775060e-01  5.02162885e-01
  3.82679587e-01  3.52014210e-01  2.71757869e-01  1.99151294e-01
  5.63066956e-01  7.02735698e-02  5.85565629e-01  5.59919890e-01
  3.81079790e-01  5.25716641e-01  1.38019437e-01  8.04843862e-01
  5.04121483e-01  5.46130111e-01  4.79151572e-01  2.51700088e-01
  3.16075061e-01  9.16848513e-02 -2.404917

(2, 100000)
const    0.593229
x1      -0.121348
x2       0.091552
x3      -0.091656
x4      -0.091016
dtype: float64
[0.06115401 0.48316617 0.85337373 ... 1.19376736 1.18084178 0.90626703]
(2, 10000)
const    0.446934
x1       0.014300
x2       0.017184
x3      -0.029034
x4       0.006933
dtype: float64
[0.62460439 0.21455744 0.86766811 ... 0.25276199 0.70806186 0.30894202]


In [63]:
a = [[1,2,3],[4,5,6]]
print(a[0][1:])
np.shape(a)

[2, 3]


(2, 3)